<a href="https://colab.research.google.com/github/talhaanwarch/Corona_Virus/blob/master/CT_scan/ensemble_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install efficientnet

### CLone github repo, to dowlonad data

In [2]:
!git clone https://github.com/UCSD-AI4H/COVID-CT.git

Cloning into 'COVID-CT'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 5443 (delta 60), reused 9 (delta 2), pack-reused 5313
Receiving objects: 100% (5443/5443), 1.09 GiB | 42.21 MiB/s, done.
Resolving deltas: 100% (351/351), done.
Checking out files: 100% (1048/1048), done.


### Extract data in one format

In [0]:
%%capture
!unzip /content/COVID-CT/Images-processed/CT_COVID.zip
!unzip /content/COVID-CT/Images-processed/CT_NonCOVID.zip

In [0]:
!mkdir images
!cp -r /content/CT_COVID/* images
!cp -r /content/CT_NonCOVID/* images


###read meta files

In [0]:
import pandas as pd
import os

In [6]:
covid=os.listdir('/content/CT_COVID')
cov=list(zip(covid,['covid']*(len(covid))))
covid_df = pd.DataFrame(cov,columns=['file','label']) 
covid_df.head()

,file,label
0,PIIS0140673620303603%3.png,covid
1,2020.03.30.20047985-p9-56%1.png,covid
2,PIIS0140673620302117_2.png,covid
3,2020.03.22.20040782-p25-1542.png,covid
4,bmj.m606.full-p4-22%1.png,covid


In [7]:
noncovid=os.listdir('/content/CT_NonCOVID')
noncov=list(zip(noncovid,['noncovid']*(len(noncovid))))
noncovid_df = pd.DataFrame(noncov,columns=['file','label']) 
noncovid_df.head()

,file,label
0,81%2.jpg,noncovid
1,583.png,noncovid
2,5%7.jpg,noncovid
3,1212.png,noncovid
4,46.jpg,noncovid


In [8]:
dataframe=pd.concat([covid_df,noncovid_df])
dataframe.head()

,file,label
0,PIIS0140673620303603%3.png,covid
1,2020.03.30.20047985-p9-56%1.png,covid
2,PIIS0140673620302117_2.png,covid
3,2020.03.22.20040782-p25-1542.png,covid
4,bmj.m606.full-p4-22%1.png,covid


In [9]:
df = pd.get_dummies(dataframe['label'])
df = pd.concat([dataframe, df], axis=1)
df=df.sample(frac=1)
df.head()


,file,label,covid,noncovid
161,2020.03.25.20043166-p17-78.png,covid,1,0
251,bmj.m606.full-p4-22%6.png,covid,1,0
85,2020.02.22.20024927-p18-66%1.png,covid,1,0
321,2020.02.25.20021568-p24-111%9.png,covid,1,0
296,1210.png,noncovid,0,1


#### TEST DATA

In [10]:
covid_test=pd.read_csv('/content/COVID-CT/Data-split/COVID/testCT_COVID.txt',header=None,squeeze=True).tolist()
noncovid_test=pd.read_csv('/content/COVID-CT/Data-split/NonCOVID/testCT_NonCOVID.txt',header=None,squeeze=True).tolist()
test=covid_test+noncovid_test
len(test)

203

In [11]:
len(test),len(covid_test),len(noncovid_test)

(203, 98, 105)

In [0]:
test_df=df[df['file'].isin(covid_test)]
train_df=df[~df['file'].isin(covid_test)]

In [13]:
len(df),len(train_df),len(test_df)

(746, 648, 98)

### Data generator

In [0]:
from keras_preprocessing.image import ImageDataGenerator
data_gen= ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=.2,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.2,
    brightness_range=[0.2, 2.0],
    )

In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf

#Train model

In [0]:
img_shape=300
batch_size=20

In [0]:
from sklearn.model_selection import KFold
import gc

from sklearn.metrics import classification_report,accuracy_score,f1_score,roc_auc_score
classification_reports=[]
kf = KFold(n_splits=5)
history=[]
accuracy=[]
f1_scores=[]
auc=[]
fold=0
for train_index, test_index in kf.split(train_df):
  train=train_df.iloc[train_index,:]
  val=train_df.iloc[test_index,:]
  print('-----------fold {}--------------'.format(fold))
  train_generator=data_gen.flow_from_dataframe(train,directory='images',
                                                      target_size=(img_shape,img_shape),
                                                      x_col="file",
                                                      y_col=['covid','noncovid'],
                                                      class_mode='raw',
                                                      shuffle=True,
                                                      batch_size=batch_size)

  val_generator=data_gen.flow_from_dataframe(val,directory='images',
                                                        target_size=(img_shape,img_shape),
                                                        x_col="file",
                                                        y_col=['covid','noncovid'],
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        batch_size=batch_size)
  
  test_generator=data_gen.flow_from_dataframe(test_df,directory='images',
                                                      target_size=(img_shape,img_shape),
                                                      x_col="file",
                                                      y_col=['covid','noncovid'],
                                                      class_mode='raw',
                                                      shuffle=False,
                                                      batch_size=batch_size)
  #train_generator.next()[0].shape,train_generator.next()[1].shape
  import efficientnet.tfkeras as efn
  model1 =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))
  model2 =efn.EfficientNetB3(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))
  ####################################################################################################
  def new_layer(model):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation="relu")(x)
    #x = Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
    predictions = Dense(2, activation="softmax")(x)
    return predictions

  ########################################################################################

  model1 = Model(inputs=model1.input, outputs=new_layer(model1))
  model2 = Model(inputs=model2.input, outputs=new_layer(model2))
  model1.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
  model2.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
########################################################################################
  model1_results = model1.fit(train_generator,epochs=25,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3, min_lr=0.000001)])
  model2_results = model2.fit(train_generator,epochs=25,
                            steps_per_epoch=train_generator.n/batch_size,
                            validation_data=val_generator,
                            validation_steps=val_generator.n/batch_size,
                            callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3, min_lr=0.000001)])
  
  
  
  
  test_generator.reset()
  y_pred1=model1.predict(test_generator,steps=test_generator.n/batch_size,verbose=1)
  y_pred2=model2.predict(test_generator,steps=test_generator.n/batch_size,verbose=1)
  y_pred=(y_pred1+y_pred2)/2
  y_pred=y_pred.round().astype(int)
  y_true=test_df.iloc[:,2::]
  classification_reports.append(classification_report(y_true, y_pred,target_names=['covid','noncovid']))#
  accuracy.append(accuracy_score(y_true,y_pred))
  auc.append(roc_auc_score(y_true,y_pred))
  f1=f1_score(y_true,y_pred,average='macro')
  print('f1 score is ', f1)
  f1_scores.append(f1)
  
  fold+=1
  del model
  tf.keras.backend.clear_session()
  gc.collect()

-----------fold 0--------------
Found 518 validated image filenames.
Found 130 validated image filenames.
Found 98 validated image filenames.
43933696/43933088 [==============================] - 2s 0us/step
Epoch 1/25
26/25 [==============================] - 32s 1s/step - loss: 0.6653 - accuracy: 0.6236 - val_loss: 0.6819 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 2/25
26/25 [==============================] - 26s 991ms/step - loss: 0.6175 - accuracy: 0.6583 - val_loss: 0.6358 - val_accuracy: 0.7000 - lr: 1.0000e-04
Epoch 3/25
26/25 [==============================] - 26s 986ms/step - loss: 0.5700 - accuracy: 0.7143 - val_loss: 0.6123 - val_accuracy: 0.7154 - lr: 1.0000e-04
Epoch 4/25
26/25 [==============================] - 26s 986ms/step - loss: 0.5317 - accuracy: 0.7355 - val_loss: 0.5601 - val_accuracy: 0.7154 - lr: 1.0000e-04
Epoch 5/25
26/25 [==============================] - 26s 988ms/step - loss: 0.4545 - accuracy: 0.7799 - val_loss: 0.5641 - val_accuracy: 0.7077 - lr: 1.0000e

In [0]:
accuracy


[0.8533333333333334,
 0.8859060402684564,
 0.8926174496644296,
 0.8791946308724832,
 0.8791946308724832]

In [0]:
import numpy as np
print(np.mean(accuracy))

0.8780492170022371


In [0]:
f1_scores

[0.8432750759878418,
 0.8858854800198226,
 0.8925739005046864,
 0.8789274106175515,
 0.8791456380677721]

In [0]:
np.mean(f1_scores)

0.8759615010395349

In [0]:
for i in range(5):
  print('--------------FOLD {}------------------'.format(i))
  print(classification_reports[i])

--------------FOLD 0------------------
              precision    recall  f1-score   support

       covid       0.85      0.76      0.80        59
    noncovid       0.86      0.91      0.88        91

   micro avg       0.85      0.85      0.85       150
   macro avg       0.85      0.84      0.84       150
weighted avg       0.85      0.85      0.85       150
 samples avg       0.85      0.85      0.85       150

--------------FOLD 1------------------
              precision    recall  f1-score   support

       covid       0.87      0.91      0.89        74
    noncovid       0.90      0.87      0.88        75

   micro avg       0.89      0.89      0.89       149
   macro avg       0.89      0.89      0.89       149
weighted avg       0.89      0.89      0.89       149
 samples avg       0.89      0.89      0.89       149

--------------FOLD 2------------------
              precision    recall  f1-score   support

       covid       0.89      0.89      0.89        73
    noncovid